## crawler class 

In [92]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import sqlite3
import re

# Create a list of words to ignore
ignorewords={'the':1,'of':1,'to':1,'and':1,'a':1,'in':1,'is':1,'it':1}

class Crawler:
    # Initialize the crawler with the name of database
    def __init__(self, db_name):
        self.con = sqlite3.connect(db_name)
    
    def __del__(self):
        self.con.close()
        
    def db_commit(self):
        self.con.commit()
    
    # Axillary function for getting an entry id and adding
    # it if it's not present
    def get_entry_id(self, table, field, value, create_new=True):
        cur=self.con.execute("select rowid from %s where %s='%s'" % (table,field,value))
        res=cur.fetchone()
        if res==None:
            cur=self.con.execute("insert into %s (%s) values ('%s')" % (table,field,value))
            return cur.lastrowid
        else:
            return res[0] 
    
    # Create index for every page
    def add_to_index(self, url, soup):
        if self.is_indexed(url):
            return 
        print('Indexing %s' % url)
        
        # Get the individual words
        text = self.get_text_only(soup)
        words = self.separate_words(text)
        
        # Get the id of URL
        url_id = self.get_entry_id('urllist', 'url', url)
        
        # Link each word to this url
        for i in range(len(words)):
            word = words[i]
            if word in ignorewords:
                continue
            word_id = self.get_entry_id('wordlist','word',word)
            self.con.execute("insert into wordlocation(urlid,wordid,location) \
                             values (%d,%d,%d)" % (url_id,word_id,i))
        
    # Extract the text from an HTML page (no tags)
    def get_text_only(self, soup):
        v = soup.string
        if v==None:
            c = soup.contents
            result_text=''
            for t in c:
                sub_text = self.get_text_only(t)
                result_text += sub_text + '\n'
            return result_text
        else:
            return v.strip()
    
    # Separate words by any non_whitespace character
    def separate_words(self, text):
        splitter = re.compile('\\W*')
        return [s.lower() for s in splitter.split(text) if s!='']
    
    # Return true if the url is already indexed 
    def is_indexed(self, url):
        u = self.con.execute("select rowid from urllist where url='%s'" % url).fetchone()
        if u != None:
            v = self.con.execute("select * from wordlocation where urlid=%d" % u[0]).fetchone()
            if v != None:
                return True
        return False
    
    # Add a link between two pages
    def add_link_ref(self, urlFrom, urlTo, linkText):
        pass
    
    # Starting with a list of pages, do a breadth 
    # first search to the given depth, indexing pages 
    # as we go
    def crawl(self, pages, depth=2):
        for i in range(depth):
            print('depth %d begins' % i)
            new_pages = set()
            for page in pages:
                try:
                    c = urllib.request.urlopen(page)
                except:
                    print('Could not open %s' % page)
                    continue
                soup = BeautifulSoup(c.read(),'lxml')
                self.add_to_index(page, soup)
                
                links = soup('a')
                for link in links:
                    if ('href' in dict(link.attrs)):
                        url = urllib.parse.urljoin(page, link['href'])
                        if url.find("'")!=-1:
                            continue
                        url=url.split('#')[0]  # remove location portion
                        if url[0:4]=='http' and not self.is_indexed(url):
                            new_pages.add(url)
                        link_text = self.get_text_only(link)
                        self.add_link_ref(page, url, link_text)
                self.db_commit()
            pages = new_pages
    
    # Create the database tables
    def create_index_tables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linkid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.db_commit()

In [94]:
crawler = Crawler('searchindex.db')
crawler.create_index_tables()

In [99]:
page_list = ['http://www.bilibili.com']
crawler.crawl(page_list)

depth 0 begins
depth 1 begins
Indexing http://www.bilibili.com/video/kichiku-manual_vocaloid-1.html


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:69: FutureWarning: split() requires a non-empty pattern match.


Indexing http://www.bilibili.com/video/music-original-1.html
Indexing http://www.bilibili.com/video/three-dimension-dance-1.html
Indexing http://www.bilibili.com/video/ent-Kichiku-1.html
Indexing http://www.bilibili.com/video/ent_funny_1.html
Indexing http://www.bilibili.com/video/technology.html
Indexing http://www.bilibili.com/video/ent-variety-1.html
Indexing http://www.bilibili.com/video/fashion.html
Indexing http://www.bilibili.com/video/tech-popular-science-1.html
Indexing http://www.bilibili.com/video/tech-future-other-1.html
Indexing http://www.bilibili.com/video/music-coordinate-1.html
Indexing http://www.bilibili.com/video/ent-sports-1.html
Indexing http://www.bilibili.com/video/douga-kichiku-1.html
Indexing http://www.bilibili.com/video/dance.html
Indexing http://www.bilibili.com/video/douga-else-1.html
Indexing http://www.bilibili.com/video/ad-ad-1.html
Indexing http://www.bilibili.com/video/tech-fun-1.html
Indexing http://www.bilibili.com/video/ent-handmake-1.html
Indexing

In [45]:
[row for row in crawler.con.execute('select rowid,urlid from wordlocation where wordid=2')]

[(2, 1),
 (760, 3),
 (1086, 9),
 (1360, 12),
 (1740, 13),
 (8343, 18),
 (8763, 20),
 (9303, 23),
 (9718, 27),
 (9812, 28),
 (10296, 37),
 (10301, 38),
 (12701, 41),
 (13481, 43),
 (13767, 46),
 (14131, 48),
 (14170, 49),
 (16238, 53),
 (16560, 54),
 (17984, 74),
 (18400, 81),
 (24293, 82),
 (24651, 88),
 (25413, 92),
 (26867, 100),
 (28355, 107),
 (28924, 111),
 (29651, 121),
 (29745, 123)]

## Querying 

In [116]:
class Searcher:
    def __init__(self, db_name):
        self.con = sqlite3.connect(db_name)
    
    def __del__(self):
        self.con.close()
        
    def get_match_rows(self, q):
        # Strings to build query
        field_list = 'w0.urlid'
        table_list = ''
        clause_list = ''
        word_ids = []
        
        # Split words by spaces
        words = q.split(' ')
        table_number = 0
        
        for word in words:
            # get id of the word
            word_row = self.con.execute("select rowid from wordlist where word = '%s'" % word).fetchone()
            if word_row != None:
                word_id = word_row[0]
                word_ids.append(word_id)
                if table_number>0:
                    table_list += ','
                    clause_list += ' and '
                    clause_list+='w%d.urlid=w%d.urlid and ' % (table_number-1,table_number)
                field_list+=',w%d.location' % table_number
                table_list+='wordlocation w%d' % table_number      
                clause_list+='w%d.wordid=%d' % (table_number,word_id)
                table_number+=1
                
        # Create the query from the separate parts
        full_query='select %s from %s where %s' % (field_list,table_list,clause_list)
        print (full_query)
        cur=self.con.execute(full_query)
        rows=[row for row in cur]
        
        return rows, word_ids
    
    def get_scored_list(self, rows, word_ids):
        total_scores = dict([(row[0], 0) for row in rows])
        
        # score function
        weighs = [(0.0, self.frequency_score(rows)),
                 (1.0, self.location_score(rows)),
                 (0.0, self.distance_score(rows)),
                 (0.0, self.inbound_link_score(rows))]
        
        for (weight, scores) in weighs:
            for url in total_scores:
                total_scores[url] += weight*scores[url]
                
        return total_scores
    
    def get_url_name(self, id):
        return self.con.execute("select url from urllist where rowid = %d" % id).fetchone()[0]
    
    def query(self, q):
        rows, word_ids = self.get_match_rows(q)
        scores = self.get_scored_list(rows, word_ids)
        ranked_scores = sorted([(score, url) for (url, score) in scores.items()], reverse=1)
        for (score, url_id) in ranked_scores[0:10]:
            print('%f\t%s' % (score, self.get_url_name(url_id)))
            
    def normalize_scores(self, scores, small_is_better=0):
        v_small = 0.00001 # Avoid division by zero errors
        if small_is_better:
            min_score = min(scores.values())
            return dict([(u, float(min_score)/max(v_small,1)) for (u, l) in scores.items()])
        else:
            max_score = max(scores.values())
            if max_score == 0:
                max_score = v_small
            return dict([(u, float(c)/max_score) for (u, c) in scores.items()])
        
    def frequency_score(self, rows):
        counts = dict([(row[0], 0) for row in rows])
        for row in rows:
            counts[row[0]] += 1
        return self.normalize_scores(counts)
    
    def location_score(self, rows):
        locations = dict([(row[0], 0) for row in rows])
        for row in rows:
            loc = sum(row[1:])
            if loc<locations[row[0]]:
                locations[row[0]] = loc
        return self.normalize_scores(locations, small_is_better=1)
    
    def distance_score(self, rows):
        # if there's only one word, everyone wins!
        if len(rows[0])<2:
            return dict([(row[0], 1.0) for row in rows])
        
        # Initial dict with large values
        min_distance = dict([(row[0], 1000000) for row in rows])
        
        for row in rows:
            dist = sum([abs(row[i]-row[i-1]) for i in range(2, len(row))])
            if dist<min_distance[row[0]]:
                min_distance[row[0]] = dist
        return self.normalize_scores(min_distance, small_is_better=1)
    
    def inbound_link_score(self, rows):
        unique_urls = dict([(row[0], 1) for row in rows])
        inbound_count = dict([(u,self.con.execute('select count(*) from link where toid=%d' % u)
                                   .fetchone()[0]) for u in unique_urls])
        return self.normalize_scores(inbound_count) 

In [117]:
e = Searcher('searchindex.db')
e.query('bilibili 直播')

select w0.urlid,w0.location,w1.location from wordlocation w0,wordlocation w1 where w0.wordid=5 and w0.urlid=w1.urlid and w1.wordid=24
0.000000	http://www.bilibili.com/html/friends-links.html
0.000000	https://account.bilibili.com/login
0.000000	http://live.bilibili.com/single
0.000000	https://www.bilibili.com/register
0.000000	http://live.bilibili.com/subject
0.000000	http://bangumi.bilibili.com/movie/
0.000000	http://h.bilibili.com/
0.000000	http://game.bilibili.com
0.000000	http://www.bilibili.com/html/contact.html
0.000000	http://live.bilibili.com/mobile


In [103]:
crawler.con.execute('select count(*) from urllist').fetchone()[0]

124